In [3]:
from datetime import datetime, timezone


def date_to_utc_timestamp(c_date):
    return int(c_date.replace(tzinfo=timezone.utc).timestamp())


def utc_timestamp_to_date(ts):
    return datetime.fromtimestamp(ts, tz=timezone.utc)


start_date = datetime(2023, 5, 1)
print(date_to_utc_timestamp(start_date))
# print(utc_timestamp_to_date(1626480000))

1682899200


In [11]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

# 0.05% WBTC/ETH pool
LOW_POOL_ID = "0x4585fe77225b41b697c938b018e2ac67ac5a20c0"
# 0.3% WBTC/ETH pool
HIGH_POOL_ID = "0xcbcdf9626bc03e24f779434178a73a0b4bad62ed"


def get_pool_trade_data(pool_id, start_date):
    pool_query = """
    query poolDayDatas($pool_id: ID!, $start_time: Int!,){
        poolDayDatas(
            where: { pool: $pool_id, date_gt: $start_time }
        orderBy: date
        orderDirection: asc
        ) {
            date
            # in range liquidity at end of period
            liquidity
            # current price tracker at end of period
            sqrtPrice
            # price of token0 - derived from sqrtPrice
            tick
            # tvl derived in USD at end of period
            tvlUSD
            # volume in token0
            volumeToken0
            # volume in token1
            volumeToken1
            # volume in USD
            volumeUSD
            # fees in USD
            feesUSD
            # number of transactions during period
            txCount
            # opening price of token0
            open
            # high price of token0
            high
            # low price of token0
            low
            # close price of token0
            close
            }
    }
    """

    client = Client(
        transport=RequestsHTTPTransport(
            url='https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3',
            verify=True,
            retries=5,
        ))

    start_timestamp = date_to_utc_timestamp(start_date)
    variables = {"pool_id": pool_id, "start_time": start_timestamp}
    response = client.execute(gql(pool_query), variable_values=variables)
    trade_data = response["poolDayDatas"]
    trade_all_data = trade_data

    while len(trade_data) > 0:
        print(utc_timestamp_to_date(start_timestamp))
        print(len(trade_all_data))
        start_timestamp = trade_data[-1]['date']
        variables = {"pool_id": pool_id, "start_time": start_timestamp}
        response = client.execute(gql(pool_query), variable_values=variables)
        trade_data = response["poolDayDatas"]
        trade_all_data += trade_data

    return trade_all_data


# the earliest data is from 2021-05-01
start_date = datetime(2021, 5, 1)
pool_data = get_pool_trade_data(LOW_POOL_ID, start_date)

2021-05-01 00:00:00+00:00
100
2021-08-12 00:00:00+00:00
200
2021-11-20 00:00:00+00:00
300
2022-02-28 00:00:00+00:00
400
2022-06-08 00:00:00+00:00
500
2022-09-16 00:00:00+00:00
600
2022-12-25 00:00:00+00:00
700
2023-04-04 00:00:00+00:00
732


In [4]:
import pandas as pd

uniswap_df = pd.DataFrame(pool_data)
uniswap_df.head()

,date,liquidity,sqrtPrice,tick,tvlUSD,volumeToken0,volumeToken1,volumeUSD,feesUSD,txCount,open,high,low,close
0,1620172800,0,0,None,0,0,0,0,0,2,0,0,0,0
1,1620259200,47802696926688,31809195401932648817609369003249547,258071,5146.736482364860299518079151516243,0.01955837,0.316827345146869434,1102.031018311199696439388175003343,0.5510155091555998482196940875016715,16,0.05998654715076262372902751818670286,0.06243654451824971693450264979198528,0.05998654715076262372902751818670286,0.05998654715076262372902751818670286
2,1620345600,42425055535734978,32187992991029178621379247853387443,258308,2073400.162975270801081924615102077,68.18328357,1120.626959335943537703,3922263.61944071584148036405495539,1961.131809720357920740182027477691,227,0.06203743019377815473808300194149325,0.06203743019377815473808300194149325,0.06010084251900176152287862020188552,0.06203743019377815473808300194149325
3,1620432000,4036037627994723,30814946235335421635470353514297570,257436,1980521.049385023017241543520869319,126.96928506,2072.7905681432721063,7443618.326347876067605645380688426,3721.809163173938033802822690344206,257,0.06058403976932998513307639824064359,0.06669932160003833798769060727416865,0.06004384605589742079211220851145031,0.06058403976932998513307639824064359
4,1620518400,672440555897057,30550018995856811291234861690516572,257263,739735.1727825094073093447014429385,10.91169152,163.331111671204485144,634883.3690065130877645983992275799,317.4416845032565438822991996137902,99,0.06609184203913069571728478548803438,0.06830342385985722044198599944892239,0.0653848260519338954289133091895779,0.06609184203913069571728478548803438


In [5]:
uniswap_df.to_csv("../data/wbtc_eth_0.05_uniswap_data.csv", index=False)

In [10]:
pool_data1 = get_pool_trade_data(HIGH_POOL_ID, start_date)
uniswap_df1 = pd.DataFrame(pool_data1)
uniswap_df1.to_csv("../data/wbtc_eth_0.3_uniswap_data.csv", index=False)

2021-05-01 00:00:00+00:00
100
2021-08-11 00:00:00+00:00
200
2021-11-19 00:00:00+00:00
300
2022-02-27 00:00:00+00:00
400
2022-06-07 00:00:00+00:00
500
2022-09-15 00:00:00+00:00
600
2022-12-24 00:00:00+00:00
700
2023-04-03 00:00:00+00:00
733


In [8]:
uniswap_df1.head()

,date,liquidity,sqrtPrice,tick,tvlUSD,volumeToken0,volumeToken1,volumeUSD,feesUSD,txCount,open,high,low,close
0,1620172800,3990726651924459,1339927351757144388712475515525518,194725,15990.5025439940886751925509600602,3695.686153,1.053733492783073142,3732.094428775605445003159429854333,1.866047214387802722501579714927167,14,0,3525.673097867570086322273753776014,0,0
1,1620259200,68047172598720294,1339910162090425200965669972282089,194725,513871.4159407929723419764491435211,1803816.327163,516.145087582265653465,1804100.360824469382992833462819919,902.0501804122346914964167314099573,906,3519.716693090278316252338136457059,3613.912924179666876274469797086365,3395.383086192493988252557527608186,3519.716693090278316252338136457059
2,1620345600,1126614144148477771,1342329450130962182714220163061212,194761,2749923.824139887875098979243647372,24132900.806355,6912.410379553819027454,24132962.61343503546513328194485803,12066.48130671751773256664097242894,1979,3492.800580689646176381188242378386,3577.139147665005703027959377071504,3367.326464953274994888859401816482,3492.800580689646176381188242378386
3,1620432000,87209657035771435,1271215747025067696231868854587672,193672,1576917.792811014489888423519154903,21335972.622553,5949.015215539578664786,21333664.52815374778086557562483543,10666.83226407687389043278781241774,1372,3448.215489988570176415924695536605,4065.42977939784842474404892687385,3448.215489988570176415924695536605,3448.215489988570176415924695536605
4,1620518400,191200334840491900,1267131081268782544641468500372448,193608,1558604.980082295860302258155946276,6330422.177312,1631.843705113879928786,6330963.787682867735399760167184382,3165.481893841433867699880083592182,1097,3886.561269519253200911892003949243,3998.817352529580010161614476694996,3739.03329448136685809450955097815,3886.561269519253200911892003949243


In [1]:
# fetch arb swap data

from datetime import datetime
from utils import utils


start_timestamp = utils.date_to_utc_timestamp(datetime(2023, 5, 23, 0, 0))
end_timestamp = utils.date_to_utc_timestamp(datetime(2023, 5, 28, 0, 0))

In [2]:
# ARB graphQL
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport


arb_query = """
query {
    swaps(skip: %d, orderBy: timestamp, orderDirection: asc,
        where: { pool: "0xc31e54c7a869b9fcbecc14363cf510d1c41fa443", timestamp_gte: %d }){
            id
            account {
                id
                positionCount
                openPositionCount
                closedPositionCount
                depositCount
                withdrawCount
                swapCount
            }
            gasLimit
            gasPrice
            amountIn
            amountInUSD
            amountOut
            amountOutUSD
            tick
            timestamp
            tokenIn {
                symbol
            }
            tokenOut {
                symbol
            }
    }
}
"""


client = Client(
    transport=RequestsHTTPTransport(
        url='https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-arbitrum',
        verify=True,
        retries=5,
    ))

# # variables = { "num_skip": 0, "start_timestamp": 1684460940 }
# # response = client.execute(gql(arb_query), variable_values=variables)
response = client.execute(gql(arb_query % (0, start_timestamp)))
print(len(response["swaps"]))

100


In [4]:
result = response['swaps']
id_set = set()
print_set = set()
PRINT_GAP = 1000

for item in result:
    id_set.add(item["id"])

# more than 100 swap transactions happened in a second, will need to change skip count
skip = 0
last_cnt = 0
while int(result[-1]["timestamp"]) <= end_timestamp:
    if skip == 1000:
        break
    last_cnt = len(result)

    print_cnt = int(len(result) / PRINT_GAP)
    if print_cnt not in print_set:
        print_set.add(print_cnt)
        print(datetime.now(), "result cnt:", len(result), "id cnt:", len(id_set), "skip: ", skip)
        print("Current time: ", utils.utc_timestamp_to_date(int(result[-1]["timestamp"])))

    current_timestamp = int(result[-1]["timestamp"])
    query = arb_query % (skip, current_timestamp)
    try:
        response = client.execute(gql(query))

        swap_data = response["swaps"]
        if len(swap_data) == 0:
            break
        for item in swap_data:
            if int(item["timestamp"]) > end_timestamp:
                break
            elif item["id"] in id_set:
                continue
            else:
                result.append(item)
                id_set.add(item["id"])

        if last_cnt == len(result):
            skip += 100
        else:
            skip = 0
    except Exception as ex:
        print("Fetching swap data error", ex)

2023-05-28 18:00:48.721025 result cnt: 100 id cnt: 100 skip:  0
Current time:  2023-05-23 00:32:21+00:00
2023-05-28 18:01:31.294167 result cnt: 1069 id cnt: 1069 skip:  0
Current time:  2023-05-23 01:34:29+00:00
2023-05-28 18:02:11.946026 result cnt: 2051 id cnt: 2051 skip:  0
Current time:  2023-05-23 02:47:25+00:00
2023-05-28 18:02:52.746591 result cnt: 3024 id cnt: 3024 skip:  0
Current time:  2023-05-23 03:20:59+00:00
2023-05-28 18:03:32.475233 result cnt: 4010 id cnt: 4010 skip:  0
Current time:  2023-05-23 04:15:54+00:00
2023-05-28 18:04:17.051785 result cnt: 5093 id cnt: 5093 skip:  0
Current time:  2023-05-23 06:04:21+00:00
2023-05-28 18:04:57.272286 result cnt: 6077 id cnt: 6077 skip:  0
Current time:  2023-05-23 07:20:10+00:00
2023-05-28 18:05:37.132056 result cnt: 7050 id cnt: 7050 skip:  0
Current time:  2023-05-23 08:48:01+00:00
2023-05-28 18:06:17.536927 result cnt: 8034 id cnt: 8034 skip:  0
Current time:  2023-05-23 10:45:52+00:00
2023-05-28 18:06:57.690793 result cnt: 

In [19]:
import pandas as pd
from copy import deepcopy


arb_result = deepcopy(result)
for item in arb_result:
    item["transaction"] = item["id"]
    item["trader"] = item["account"]["id"]
    item["traderPositionCount"] = item["account"]["positionCount"]
    item["traderOpenPositionCount"] = item["account"]["openPositionCount"]
    item["traderClosedPositionCount"] = item["account"]["closedPositionCount"]
    item["traderDepositCount"] = item["account"]["depositCount"]
    item["traderWithdrawCount"] = item["account"]["withdrawCount"]
    item["traderSwapCount"] = item["account"]["swapCount"]
    item["tokenIn"] = item["tokenIn"]["symbol"]
    item["tokenOut"] = item["tokenOut"]["symbol"]
    item.pop("account")

arb_uniswap_df = pd.DataFrame(arb_result)
arb_uniswap_df.to_csv("../data/arb_swap_0523_0528.csv", index=False)